# How to upload and publish a research product on Zenodo through the Zenodo REST API

**Zenodo** (https://zenodo.org) is an open repository for all scholarship, enabling researchers from all disciplines to share and preserve their research outputs, regardless of size or format. Free to upload and free to access, Zenodo makes scientific outputs of all kinds **citable**, **shareable** and **discoverable** for the long term.

Zenodo provides a **REST API** which currently supports:

 - **Deposit** — upload and publishing of research outputs (identical to functionality available in the user interface).
 - **Records** — search published records.
 - **Files** — download/upload of files
 
The base URL of the API is https://zenodo.org/api/ (or https://sandbox.zenodo.org/api/ for testing purposes).

Besides the community related to the EGI-ACE project (https://zenodo.org/communities/egi-ace/) - which is the EU-funded project supporting activities around Data Spaces - the  ENES Data Space has its own community on Zenodo (https://zenodo.org/communities/enes-data-space).

In this way, you can upload your resources and share them with the community, thus fostering opportunities for collaboration and promoting a more sustainable, effective, and FAIR use of data.

## *Depositions* entity
The Deposition resource is used for uploading and editing records on Zenodo.

 - List all the depositions for the currently authenticated user
    
    `GET /api/deposit/depositions`
    
 - Create a new deposition resource
    
    `POST /api/deposit/depositions`
    
 - Retrieve a single deposition resource
    
    `GET /api/deposit/depositions/:id`
  
 - Update an existing deposition resource
    
    `PUT /api/deposit/depositions/:id`
     
 - Delete an existing deposition resource. Note, only unpublished depositions may be deleted.
    
    `DELETE /api/deposit/depositions/:id`

## Deposition *files*
The Deposition file resource is used for uploading and editing files of a deposition on Zenodo.

 - List all the deposition files for a given deposition
    
    `GET /api/deposit/depositions/:id/files`
    
 - Upload a new file
   
   `POST /api/deposit/depositions/:id/files`
    
 - Sort the files for a deposition. By default, the first file is shown in the file preview
   
   `PUT /api/deposit/depositions/:id/files`
  
 - Retrieve a single deposition file
   
   `GET /api/deposit/depositions/:id/files/:file_id`
     
 - Update a deposition file resource. Currently the only use is renaming an already uploaded file. If you one to replace the actual file, please delete the file and upload a new file.
   
   `PUT /api/deposit/depositions/:id/files/:file_id`
 
 - Delete an existing deposition file resource. Note, only deposition files for unpublished depositions may be deleted.
   
   `DELETE /api/deposit/depositions/:id/files/:file_id`

## Deposition *actions*

- Publish a deposition. Note, once a deposition is published, you can no longer delete it.
    
    `POST /api/deposit/depositions/:id/actions/publish`
    
 - Unlock already submitted deposition for editing
   
   `POST /api/deposit/depositions/:id/actions/edit`
    
 - Discard changes in the current editing session
   
   `POST /api/deposit/depositions/:id/actions/discard`
  
 - Create a new version of a deposition
   
   `POST /api/deposit/depositions/:id/actions/newversion`

## *Records* entity
The Records resource is used to search through published records.

- List all open access records: [here](https://developers.zenodo.org/#list36) more info about the query parameters
    
    `GET /api/records/`
    
 - Retrieve a single record
   
   `GET /api/records/:id`
    
 - Discard changes in the current editing session
   
   `POST /api/deposit/depositions/:id/actions/discard`
  
 - Create a new version of a deposition
   
   `POST /api/deposit/depositions/:id/actions/newversion`



As first step, let's import the `requests` module to handle HTTP requests.

In [ ]:
import requests

If you wish to test the API without publishing any product online, you can use the **sandbox environment** where you can test your API integration during development. The sandbox environment is available at http://sandbox.zenodo.org.

Please note that the sandbox environment can be cleaned at anytime. Also, the sandbox environment will issue test DOIs using the `10.5072` prefix instead of the Zenodo’s normal prefix `10.5281`.

In [ ]:
base_url = "https://sandbox.zenodo.org/api/"
#base_url = "https://zenodo.org/api/"

All the  APIs  require an **access token**. Create a token and replace ACCESS_TOKEN with your newly created personal access token.

**Note:** you need two different tokens for working in the sandbox and official environments.

To create a personal access token:

  - **Register** for a **Zenodo account** if you don’t already have one.
  - Go to your Applications to **create a new token**.
  - Select the **OAuth scopes** you need (you need *deposit:write* and *deposit:actions*).
  
**Note**
  - **deposit:write** ---> Grants write access to depositions, but does not allow publishing the upload.
  - **deposit:actions** ---> Grants access to publish, edit and discard edits for depositions.


In [ ]:
#ACCESS_TOKEN = '...' #official
ACCESS_TOKEN = '...' #sandbox

### STEP 1: Let’s create a new empty upload

In [ ]:
headers = {"Content-Type": "application/json"}
params = {'access_token': ACCESS_TOKEN}
r = requests.post(base_url+'/deposit/depositions',
                   params=params,
                   json={},
                   headers=headers)
r.json()

The deposition **id** represents the deposition identifier to be used in the next steps, so let's extract it from the previous json response.

In [ ]:
deposition_id = r.json()['id']
deposition_id

Similarly, to use the new files API we will need to do a PUT request to the bucket link. The bucket is a folder-like object storing the files of our record and can be found under the 'links' key in our records metadata.

In [ ]:
bucket_url = r.json()["links"]["bucket"]
bucket_url

### STEP 2: Let’s upload a new file

To upload a new file with the **old API**: `POST /api/deposit/depositions/:id/files`
 
Example:

```
filename = "TEST.txt"
path = "/home/jovyan/work/%s" % filename
data = {'name': filename}
deposition_id = r.json()['id']
files = {'file': open(path, 'rb')}
r = requests.post(base_url+'/deposit/depositions/%s/files' % deposition_id,
                   params={'access_token': ACCESS_TOKEN}, data=data,
                   files=files)
```

In this case, we are going to use the **new API**, which is significantly more perfomant and supports much larger file sizes. While the older API supports 100MB per file, the new one has no size limitation.

The target URL is a combination of the bucket link with the desired filename seperated by a slash: `PUT /api/files/:id/filename`


In [ ]:
''' New API '''
filename = "Frost_Days_2015-2100.gif"
path = "/home/jovyan/work/%s" % filename

with open(path, "rb") as fp:
    r = requests.put(
        "%s/%s" % (bucket_url, filename),
        data=fp,
        params=params,
    )
r.json()

### STEP 3: Let's add some metadata

To do so, we need to update the **metadata** field of the desosition resource (see output of step 1).

For a comprehensive description of the available metadata attributes, please refer to the **Deposit metadata** table at the following link:
https://developers.zenodo.org/#representation

Among the others, you can specify the list of **communities** you wish the deposition to appear, for example **enes-data-space**. 

         "communities": [
             { "identifier": "enes-data-space" }
         ]

The owner of the community will be notified, and can either accept or reject your request.

NOTE: If your upload is rejected, it will still be available on Zenodo, just not in the community. 

In [ ]:
import json
data = {
     'metadata': {
         'title': 'Frost Days climate index',
         'upload_type': 'other',
         'description': 'Sample upload',
         'creators': [{'name': 'Fabrizio, Antonio','affiliation': 'CMCC'}],
         'access_right': 'open',
         'license':'cc-by',
         "communities": [
             {"identifier": "enes-data-space"}
         ],
         'keywords': ["climate index", "frost days"]
     }
 }
r = requests.put(base_url+'/deposit/depositions/%s' % deposition_id,
                  params={'access_token': ACCESS_TOKEN}, data=json.dumps(data),
                  headers=headers)
r.json()

### STEP 4: We’re ready to publish

In [ ]:
r = requests.post(base_url+'/deposit/depositions/%s/actions/publish' % deposition_id,
                      params={'access_token': ACCESS_TOKEN} )

This is the corresponding json response

In [ ]:
r.json()

And this is the DOI assigned to our record

In [ ]:
record_doi = r.json()["doi"]
record_doi

In [ ]:
record_doi_url = r.json()["doi_url"]
record_doi_url

We can also get the list of our records. The response is the list of all depositions for the currently authenticated user, as described [here](https://developers.zenodo.org/#list).

In [ ]:
depositions_list = requests.get(base_url+'/deposit/depositions',
                  params={'access_token': ACCESS_TOKEN})
depositions_list.json()

We can loop through the list and access the metadata (e.g., doi_url)

In [ ]:
for d in depositions_list.json():
    print(d["doi_url"])

## Search for some open access records

As an example, we are interested in records related to the **ENES Data Space community**.

As first step, let's check if the community does exist and get its id.

In [ ]:
ACCESS_TOKEN = 'J7kaMPAhcu1EG1JX1PlhiTGuXLxBGQ0NyeqZnX9465jARs6d8YvjMEPlC9hq'

params = {'access_token': ACCESS_TOKEN, 'q':'enes-data-space'}

r = requests.get('https://zenodo.org/api/communities/',
                  params=params)

r.json()

In [ ]:
enesds_id = r.json()['hits']['hits'][0]['id']
enesds_id

Now, let's retrieve all the records related to the ENES Data Space community. To do so, we use the `communities` query argument.

In [ ]:
params = {'access_token': ACCESS_TOKEN, 'communities':enesds_id}

r = requests.get('https://zenodo.org/api/records/',
                  params=params)

r.json()

Let's extract DOI and title of the returned records.

In [ ]:
for record in r.json()['hits']['hits']:
    print('%s - %s' % (record['metadata']['doi'], record['metadata']['title']))

We can also specify a search query (`q`) using the Elasticsearch query string syntax. For example, we are interested in records about **index**.

In [ ]:
params = {'access_token': ACCESS_TOKEN, 'communities':enesds_id, 'q':'index'}

r = requests.get('https://zenodo.org/api/records/',
                  params=params)

r.json()

In [ ]:
for record in r.json()['hits']['hits']:
    print('%s - %s' % (record['metadata']['doi'], record['metadata']['title']))